In [1]:
import pandas as pd
import re

In [2]:
upbit = [
    ("2021-11-18T13:25:02", "[거래] KRW, BTC 마켓 디지털 자산 추가 (ALGO, AUDIO)"),
    ("2021-10-27T15:00:14", "[거래] KRW, BTC 마켓 디지털 자산 추가 (1INCH, AAVE, MASK)"),
    ("2021-10-15T13:00:02", "[거래] KRW, BTC 마켓 디지털 자산 추가 (SOL, MATIC, NU)"),
    ("2021-04-14T16:00:02", "[거래] BTC 마켓 디지털 자산 추가 (GRT, SNX)"),
    ("2021-04-05T15:25:11", "[거래] 원화 마켓 디지털 자산 추가 (AXS, STX, FLOW, DAWN)"),
    ("2021-03-26T15:00:05", "[거래] BTC 마켓 디지털 자산 추가 (플로우 FLOW)"),
    ("2021-03-18T17:00:03", "[거래] BTC 마켓 디지털 자산 추가 (바운스토큰 AUCTION, 던프로토콜 DAWN)"),
    ("2021-03-10T16:00:03", "[거래] BTC 마켓 디지털 자산 추가 (니어프로토콜 NEAR)"),
    ("2021-02-09T14:00:02", "[거래] 원화 마켓 디지털 자산 추가 (도지코인 DOGE)"),
    ("2021-02-09T14:00:02", "[거래] BTC 마켓 디지털 자산 추가 (스택스 STX)")
]

In [3]:
def ticker_extractor_upbit(title):
    p = re.findall('\(([^)]+)', title)
    p = p[len(p)-1].replace(' ','').split(',')
    # 한글 없애주기
    return [re.sub('[가-힣]','',t) for t in p]

In [4]:
tickers = sum([ticker_extractor_upbit(t[1]) for t in upbit],[])
print(tickers[:10])

['ALGO', 'AUDIO', '1INCH', 'AAVE', 'MASK', 'SOL', 'MATIC', 'NU', 'GRT', 'SNX']


## 바이낸스에 업비트 상장이 영향을 미치는지 연구

In [5]:
import ccxt
from datetime import datetime
import plotly.graph_objects as go

In [6]:
def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    earliest_timestamp = exchange.milliseconds() # 현재시간?
    timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe) # 초단위로 바꿈
    timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
    timedelta = limit * timeframe_duration_in_ms
    if isinstance(since, str):
        since = exchange.parse8601(since)

    all_ohlcv = []
    while True:
        fetch_since = earliest_timestamp - timedelta
        ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
        # if we have reached the beginning of history
        if ohlcv[0][0] >= earliest_timestamp:
            break
        earliest_timestamp = ohlcv[0][0]
        all_ohlcv = ohlcv + all_ohlcv
        if len(all_ohlcv) % 100000 == 0:
            print(len(all_ohlcv), symbol, 'candles in total from', exchange.iso8601(all_ohlcv[0][0]), 'to', exchange.iso8601(all_ohlcv[-1][0]))
        # if we have reached the checkpoint
        if fetch_since < since:
            break
        
    return all_ohlcv

def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    num_retries = 0
    try:
        num_retries += 1
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        # print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
        return ohlcv
    except Exception:
        if num_retries > max_retries:
            raise  # Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')

In [13]:
def Binance_Ohlc_Chart(i):
    binance = ccxt.binance()
    ticker = tickers[i]
    public_date = upbit[i][0][:-2]+'00'
    public_date = public_date.replace('T',' ')

    all_ohlcv = scrape_ohlcv(binance, 3, f'{ticker}/USDT', '1m', '2021-01-01 00:00:00Z', 1000)
    dat = pd.DataFrame(all_ohlcv, columns=['time','open','high','low','close','volume'])
    dat['time'] = [datetime.fromtimestamp(t/1000).strftime('%Y-%m-%d %H:%M:%S') for t in dat['time']]

    idx = dat.loc[dat['time']==public_date].index[0]
    # 대략 1시간 전부터 3시간후까지
    dat_filter = dat.iloc[(idx-60):(idx+180)]

    fig = go.Figure(data=go.Candlestick(
        x=dat_filter['time'],
        open=dat_filter['open'],
        high=dat_filter['high'],
        low=dat_filter['low'],
        close=dat_filter['close']))
    fig.update_layout(
        title = ticker+'/USDT Binance Chart',
        shapes = [dict(
            x0=public_date,x1=public_date,
            y0=min(dat_filter['low'])*0.98 ,y1=max(dat_filter['high'])*1.02
        )]
    )
    fig.show()

In [14]:
Binance_Ohlc_Chart(0)

100000 ALGO/USDT candles in total from 2021-09-16T21:18:00.000Z to 2021-11-25T07:57:00.000Z
200000 ALGO/USDT candles in total from 2021-07-09T13:10:00.000Z to 2021-11-25T07:57:00.000Z
300000 ALGO/USDT candles in total from 2021-05-01T02:30:00.000Z to 2021-11-25T07:57:00.000Z
400000 ALGO/USDT candles in total from 2021-02-20T15:50:00.000Z to 2021-11-25T07:57:00.000Z


In [15]:
Binance_Ohlc_Chart(1)

100000 AUDIO/USDT candles in total from 2021-09-16T21:19:00.000Z to 2021-11-25T07:58:00.000Z
200000 AUDIO/USDT candles in total from 2021-07-09T13:10:00.000Z to 2021-11-25T07:58:00.000Z
300000 AUDIO/USDT candles in total from 2021-05-01T02:30:00.000Z to 2021-11-25T07:58:00.000Z
400000 AUDIO/USDT candles in total from 2021-02-20T15:50:00.000Z to 2021-11-25T07:58:00.000Z


In [17]:
Binance_Ohlc_Chart(3)

100000 AAVE/USDT candles in total from 2021-09-16T21:20:00.000Z to 2021-11-25T07:59:00.000Z
200000 AAVE/USDT candles in total from 2021-07-09T13:10:00.000Z to 2021-11-25T07:59:00.000Z
300000 AAVE/USDT candles in total from 2021-05-01T02:30:00.000Z to 2021-11-25T07:59:00.000Z
400000 AAVE/USDT candles in total from 2021-02-20T15:50:00.000Z to 2021-11-25T07:59:00.000Z


In [19]:
Binance_Ohlc_Chart(5)

100000 SOL/USDT candles in total from 2021-09-16T21:22:00.000Z to 2021-11-25T08:01:00.000Z
200000 SOL/USDT candles in total from 2021-07-09T13:10:00.000Z to 2021-11-25T08:01:00.000Z
300000 SOL/USDT candles in total from 2021-05-01T02:30:00.000Z to 2021-11-25T08:01:00.000Z
400000 SOL/USDT candles in total from 2021-02-20T15:50:00.000Z to 2021-11-25T08:01:00.000Z


In [20]:
Binance_Ohlc_Chart(6)

100000 MATIC/USDT candles in total from 2021-09-16T21:23:00.000Z to 2021-11-25T08:02:00.000Z
200000 MATIC/USDT candles in total from 2021-07-09T13:10:00.000Z to 2021-11-25T08:02:00.000Z
300000 MATIC/USDT candles in total from 2021-05-01T02:30:00.000Z to 2021-11-25T08:02:00.000Z
400000 MATIC/USDT candles in total from 2021-02-20T15:50:00.000Z to 2021-11-25T08:02:00.000Z


In [22]:
Binance_Ohlc_Chart(8)

100000 GRT/USDT candles in total from 2021-09-16T21:24:00.000Z to 2021-11-25T08:03:00.000Z
200000 GRT/USDT candles in total from 2021-07-09T13:10:00.000Z to 2021-11-25T08:03:00.000Z
300000 GRT/USDT candles in total from 2021-05-01T02:30:00.000Z to 2021-11-25T08:03:00.000Z
400000 GRT/USDT candles in total from 2021-02-20T15:50:00.000Z to 2021-11-25T08:03:00.000Z


In [23]:
Binance_Ohlc_Chart(9)

100000 SNX/USDT candles in total from 2021-09-16T21:25:00.000Z to 2021-11-25T08:04:00.000Z
200000 SNX/USDT candles in total from 2021-07-09T13:10:00.000Z to 2021-11-25T08:04:00.000Z
300000 SNX/USDT candles in total from 2021-05-01T02:30:00.000Z to 2021-11-25T08:04:00.000Z
400000 SNX/USDT candles in total from 2021-02-20T15:50:00.000Z to 2021-11-25T08:04:00.000Z
